In [2]:
import torch
import numpy as np
from plyfile import PlyData
import matplotlib.pyplot as plt
from diff_gaussian_rasterization import rasterize_gaussians, GaussianRasterizationSettings

In [3]:
# 1. PLY 파일 로드 및 데이터 추출
ply_path = 'gs_data/syntactic.ply'
plydata = PlyData.read(ply_path)
vertex_data = plydata['vertex'].data


In [6]:
# 2. 속성 추출
# 위치 (means3D)
vertices = np.stack([vertex_data['x'], vertex_data['y'], vertex_data['z']], axis=-1)

# 노멀 (normals)
normals = np.stack([vertex_data['nx'], vertex_data['ny'], vertex_data['nz']], axis=-1)

# 색상 (colors) - f_dc_0, f_dc_1, f_dc_2 사용
colors = np.stack([vertex_data['f_dc_0'], vertex_data['f_dc_1'], vertex_data['f_dc_2']], axis=-1)

# 불투명도 (opacity)
opacity = vertex_data['opacity']

# 스케일 (scales)
scales = np.stack([vertex_data['scale_0'], vertex_data['scale_1'], vertex_data['scale_2']], axis=-1)

# 회전 (rotations) - 쿼터니언 형태로 가정
rotations = np.stack([
    vertex_data['rot_0'],
    vertex_data['rot_1'],
    vertex_data['rot_2'],
    vertex_data['rot_3']
], axis=-1)


In [5]:
print(vertices[:3])

[[ 0.2528585   0.17946313  0.42800722]
 [ 0.13811877  0.14321326  0.6148501 ]
 [-0.27099672  0.15890008 -0.5628106 ]]


In [ ]:

# 필요한 경우 추가 특성 추출 (예: f_rest_0 ~ f_rest_44)
# f_rest = np.stack([vertex_data[f'f_rest_{i}'] for i in range(45)], axis=-1)

# 3. PyTorch 텐서로 변환 및 GPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

means3D = torch.tensor(vertices, dtype=torch.float32).to(device)
normals_tensor = torch.tensor(normals, dtype=torch.float32).to(device)
colors_tensor = torch.tensor(colors, dtype=torch.float32).to(device)
opacity_tensor = torch.tensor(opacity, dtype=torch.float32).to(device)
scales_tensor = torch.tensor(scales, dtype=torch.float32).to(device)
rotations_tensor = torch.tensor(rotations, dtype=torch.float32).to(device)

# 4. 래스터화 설정 정의

# 이미지 크기
image_height = 512
image_width = 512

# 시야각 설정
fov = 60  # degrees
aspect_ratio = image_width / image_height
tan_fovx = np.tan(np.radians(fov) / 2)
tan_fovy = tan_fovx / aspect_ratio

# 카메라 매트릭스 설정 (여기서는 단위 행렬 사용)
viewmatrix = torch.eye(4, dtype=torch.float32).to(device)
projmatrix = torch.eye(4, dtype=torch.float32).to(device)

# 배경 색상 설정 (검정색)
background = torch.zeros(3, dtype=torch.float32).to(device)

# SH (Spherical Harmonics) 설정 (사용하지 않는다면 빈 텐서 전달)
sh = torch.zeros((means3D.shape[0], 9), dtype=torch.float32).to(device)
degree = 2  # SH 차수 (필요에 따라 변경)

# 카메라 위치 설정
campos = torch.tensor([0.0, 0.0, 5.0], dtype=torch.float32).to(device)

# 래스터화 설정 객체 생성
raster_settings = GaussianRasterizationSettings(
    image_height=image_height,
    image_width=image_width,
    tanfovx=tan_fovx,
    tanfovy=tan_fovy,
    bg=background,
    scale_modifier=1.0,
    viewmatrix=viewmatrix,
    projmatrix=projmatrix,
    sh_degree=degree,
    campos=campos,
    prefiltered=False,
    debug=False
)

# 5. rasterize_gaussians 함수 호출
# colors_precomp를 사용하므로 sh는 빈 텐서로 전달
means2D = torch.empty(0).to(device)  # 2D 위치는 빈 텐서로 전달
cov3D_precomp = torch.empty(0).to(device)  # 미리 계산된 공분산이 없다면 빈 텐서 전달

color_image, radii = rasterize_gaussians(
    means3D=means3D,
    means2D=means2D,
    sh=sh,
    colors_precomp=colors_tensor,
    opacities=opacity_tensor,
    scales=scales_tensor,
    rotations=rotations_tensor,
    cov3Ds_precomp=cov3D_precomp,
    raster_settings=raster_settings
)

# 6. 결과 시각화
color_image_cpu = color_image.cpu().detach().numpy()
color_image_rgb = color_image_cpu[:, :, :3]  # RGB 채널 추출

plt.imshow(np.clip(color_image_rgb, 0, 1))
plt.axis('off')
plt.show()